In [1]:
# ! wget https://github.com/alexeygrigorev/minsearch/raw/refs/heads/main/minsearch.py

In [2]:
# import minsearch

In [3]:
# Alternate way - directly install minsearch
# ! pip install minsearch

## Retrieval and Search

In [4]:
import minsearch

/workspaces/llm-zoomcamp-2025/01-intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [5]:
import json

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
# docs_raw

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [11]:
q = 'the course has already started, can I still enroll?'

In [12]:
index.fit(documents)

In [13]:
import os
import sys
from dotenv import load_dotenv

from litellm import completion
from typing import List, Dict

from openai import OpenAI

In [14]:
# Load environment variables and set API key
load_dotenv()

True

In [15]:
client = OpenAI()

In [16]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"Whether you can enroll in a course that's already started depends on the policies of the institution or platform offering the course. Here are a few steps you can take:\n\n1. **Check the Course Page:** Look for any information about late enrollment, which might be available under the course description or FAQ sections.\n\n2. **Contact the Instructor or Administration:** Reach out to the course instructor or the administrative office. They might be able to provide an exception or additional information on late enrollment options.\n\n3. **Look for Enrollment Dates:** Verify if the course has a specific add/drop period. Some institutions allow students to join classes within the first week or two after they start.\n\n4. **Online Platforms:** If the course is offered online, platforms like Coursera, edX, or Udemy often allow you to start courses at any time, even if there is a recommended start date.\n\n5. **Consider Catching Up:** If you are allowed to enroll late, ensure that you can co

In [17]:
# Using LiteLLM

messages=[{"role": "user", "content": q}]

response = completion(
    model="openai/gpt-4o",
    messages=messages,
    # max_tokens=1024
)

response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on the specific policies of the institution or platform offering the course. Here are a few steps you can take to find out:\n\n1. **Check the Enrollment Policy:** Visit the course's webpage or the institution's website to see if they have any information about late enrollment or registration deadlines.\n\n2. **Contact the Instructor or Administration:** Reach out directly to the course instructor or the admissions office to inquire about the possibility of late enrollment. They may be able to make exceptions or offer guidance.\n\n3. **Consider Auditing the Course:** Some institutions allow students to audit courses, which means you can attend classes and access materials without officially enrolling or receiving credit.\n\n4. **Look for Online Options:** If the course is part of an online platform, check if there is a self-paced version or if late enrollments are accepted with the option to catch up on missed co

# Index

In [18]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5
)

In [19]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [20]:
# Add filtering

boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [21]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generating answers with LLM

In [24]:
# Response directly from LLM only
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [26]:
response.choices[0].message.content

"The possibility of enrolling in a course that has already started depends on several factors such as the institution's policies, availability of spots, and the specific course in question. Here are some steps you can take:\n\n1. **Contact the Institution**: Reach out to the admissions office or the course instructor to inquire about late enrollment policies.\n\n2. **Check Online**: Often, institutions provide information about enrollment deadlines and procedures on their websites.\n\n3. **Consider Open Enrollment**: Some courses, especially online ones, might allow rolling admissions or have flexible start dates.\n\n4. **Assess Catch-Up Requirements**: If late enrollment is possible, find out what will be required for you to catch up on missed material.\n\n5. **Financial Implications**: Inquire if there are any financial implications, such as late fees, associated with late enrollment.\n\nMake sure to act quickly, as the longer you wait, the more challenging it could be to catch up on

In [37]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [38]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [39]:
context

"section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n\nsection: General course-related questions\nquestion: Course - When will the course start?\nanswer: The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course 

In [41]:
prompt_template.format(question=q, context=context).strip()

"You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: the course has already started, can I still enroll?\n\nCONTEXT: section: General course-related questions\nquestion: Course - Can I still join the course after the start date?\nanswer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - Can I follow the course after it finishes?\nanswer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.\n\nsection: General 

In [42]:
prompt = prompt_template.format(question=q, context=context).strip()

In [43]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related q

In [45]:
# Retrieval augmented generation

response = client.chat.completions.create(
    model = 'gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'Yes, even if the course has already started, you can still enroll and submit the homeworks. However, be aware of the deadlines for turning in the final projects, and avoid leaving everything for the last minute.'

## Let's make code modular and reusable

In [50]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [51]:
# Test
search("how do I run kafka?")

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [56]:
query = 'how do I run kafka?'

search_results = search(query)

In [57]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [58]:
# Test
build_prompt(query, results)

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: how do I run kafka?\n\nCONTEXT: \nsection: Module 6: streaming with kafka\nquestion: Java Kafka: How to run producer/consumer/kstreams/etc in terminal\nanswer: In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nsection: Module 6: streaming with kafka\nquestion: Module “kafka” not found when trying to run producer.py\nanswer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you\'ll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on Ma

In [59]:
prompt = build_prompt(query, results)

In [60]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [61]:
# Test

answer = llm(prompt)
answer

'To run Kafka in a terminal, you can execute the following command in your project directory:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your jar file.'

In [63]:
# Another example
query = 'Course just started sometime back. Can I still join?'

search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

answer

'Yes, you can still join the course even if it has already started. You are eligible to submit the homework assignments, but keep in mind that there will be deadlines for turning in the final projects, so try not to leave everything for the last minute.'

In [64]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [65]:
# Test
query = 'how do I run Grafana?'
rag(query)

"The provided CONTEXT does not contain information about how to run Grafana. If you need specific instructions or details about running Grafana, it might be helpful to refer to Grafana's official documentation or resources related to the course that may not be covered in the provided FAQs."

In [66]:
query = 'Can I run course environment setup locally? I do not have cloud?'
rag(query)

"Yes, you can run the course environment setup locally. It's up to you which platform and environment you use for the course. GitHub Codespaces or a GCP VM are just possible options, but you can complete the entire course from your laptop without using the cloud."

## Cleaned code

In [67]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [68]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [69]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [70]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [72]:
# Test
query = 'Course just started sometime back. Can I still join?'
rag(query)

"Yes, you can still join the course even if it has already started. You are eligible to submit homeworks, but keep in mind that there will be deadlines for turning in the final projects, so it's advisable not to procrastinate."

In [73]:
rag('Course just started sometime back. Can I still join?')

'Yes, you can still join the course even if it has already started. You are eligible to submit the homeworks, but make sure to be aware of the deadlines for turning in the final projects to avoid leaving everything for the last minute.'

## Use Elasticsearch (replace minsearch)

1. Run elasticsearch in docker using commands from `commands.md`

In [80]:
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

In [75]:
es_client = Elasticsearch('http://localhost:9200') 

In [76]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [77]:
# Index documents with elasticsearch

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [81]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [83]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [84]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [85]:
# Test
query = 'I just disovered the course. Can I still join it?'
rag(query)

"Yes, you can still join the course even if you just discovered it. You are eligible to submit the homework without registering. However, keep in mind that there will be deadlines for submitting the final projects, so it's important not to leave everything until the last minute."

In [86]:
query = 'How to run kafka?'
rag(query)

'To run Kafka, if you are using a Java project, you should execute a class as follows: \n\nNavigate to your project directory and run the command:\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nThis will run the Java-based Kafka producer or consumer in the terminal. Replace `<jar_name>` with the actual name of your JAR file.'